In [46]:
pip install sqlparse

Note: you may need to restart the kernel to use updated packages.


In [47]:
query="""SELECT
  a.first_name,
  a.last_name,
  s.sales_amount
FROM artists a
JOIN (
    SELECT artist_id, SUM(sales_price) AS sales
    FROM sales s
    GROUP BY artist_id
  ) AS artist_sales 
  ON artists.id = artist_sales.artist_id;"""

In [48]:
import sqlparse
def find_selected_columns(query) -> list[str]:
    tokens = sqlparse.parse(query)[0].tokens
    found_select = False
    for token in tokens:
        if found_select:
            if isinstance(token, sqlparse.sql.IdentifierList):
                return [
                    col.value.split(" ")[-1].strip("`").rpartition('.')[-1]
                    for col in token.tokens
                    if isinstance(col, sqlparse.sql.Identifier)
                ]
        else:
            found_select = token.match(sqlparse.tokens.Keyword.DML, ["select", "SELECT"])
    raise Exception("Could not find a select statement. Weired query :)")

In [49]:
import sqlparse
def find_selected_alias(query) -> list[str]:
    tokens = sqlparse.parse(query)[0].tokens
    found_select = False
    for token in tokens:
        if found_select:
            if isinstance(token, sqlparse.sql.IdentifierList):
                return [
                    col.value.split(" ")[-1].strip("`").rpartition('.')[0]
                    for col in token.tokens
                    if isinstance(col, sqlparse.sql.Identifier)
                ]
        else:
            found_select = token.match(sqlparse.tokens.Keyword.DML, ["select", "SELECT"])
    raise Exception("Could not find a select statement. Weired query :)")

In [50]:
alias=find_selected_alias(query)
alias

['a', 'a', 's']

In [51]:
columns=find_selected_columns(query)
columns


['first_name', 'last_name', 'sales_amount']

In [52]:
import sqlparse, re
def get_tables(d):
    f = False
    for i in getattr(d, 'tokens', []):
       if isinstance(i, sqlparse.sql.Token) and i.value.lower() == 'from':
          f = True
       elif isinstance(i, (sqlparse.sql.Identifier, sqlparse.sql.IdentifierList)) and f:
          f = False
          if not any(isinstance(x, sqlparse.sql.Parenthesis) or 'select' in x.value.lower() for x in getattr(i, 'tokens', [])):
             fr = ''.join(str(j) for j in i if j.value not in {'as', '\n'})
             for t in re.findall('(?:\w+\.\w+|\w+)\s+\w+|(?:\w+\.\w+|\w+)', fr):
                yield {'table':(t1:=t.split())[0], 'alias':None if len(t1) < 2 else t1[-1]}
       yield from get_tables(i)
    
lookup1= list(get_tables(sqlparse.parse(query)[0]))

print(lookup1)

[{'table': 'artists', 'alias': 'a'}, {'table': 'sales', 'alias': 's'}]


In [53]:
import pandas as pd
df = pd.DataFrame(lookup1)

In [54]:
df

,table,alias
0,artists,a
1,sales,s


In [55]:
df_final = pd.DataFrame(
    {'Output_columns': columns,
     'alias': alias
    })

In [56]:
df_final

,Output_columns,alias
0,first_name,a
1,last_name,a
2,sales_amount,s


In [57]:
df_final2 = pd.merge(df_final, 
                      df, 
                      on ='alias', 
                      how ='inner')

In [58]:
df_final2

,Output_columns,alias,table
0,first_name,a,artists
1,last_name,a,artists
2,sales_amount,s,sales


In [59]:
df_final3 = df_final2[['Output_columns', 'table']]
df_final3

,Output_columns,table
0,first_name,artists
1,last_name,artists
2,sales_amount,sales
